# ITCS 6162: Data Mining - Programming Assignment

**In this assignment, you will explore data analysis, recommendation algorithms, and graph-based techniques using the MovieLens dataset. Your tasks will range from basic data exploration to advanced recommendation models, including:**
- Data manipulation with pandas
- User-item collaborative filtering
- Similarity-based recommendation models
- A Pixie-inspired Graph-based recommendation using adjacency lists with weighted random walks (without using NetworkX)


#### **Dataset Files:**
- **`u.data`**: User-movie ratings (`user_id  movie_id  rating  timestamp`)
- **`u.item`**: Movie metadata (`movie_id | title | release date | IMDB_website`)
- **`u.user`**: User demographics (`user_id | age | gender | occupation | zip_code`)

## **Part 1: Exploring and Cleaning Data**

### Inspecting the Dataset Format

The dataset is not in a traditional CSV format. To examine its structure, use the following shell command to display the first 10 lines of the file:

```sh
!head <file_name>


**In the cells given below. Write the code to read the files.**

In [1]:
#importing libraries
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity  # For computing user similarity

In [2]:
# u.data
#data = pd.read_csv("/content/u.data", sep='\t', header=None, names=["user_id", "movie_id", "rating", "timestamp"], index_col=False)
!head "/content/u.data"

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013


In [3]:
# u.item
!head "/content/u.item"

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

In [4]:
# u.user

!head "/content/u.user"

1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067
4|24|M|technician|43537
5|33|F|other|15213
6|42|M|executive|98101
7|57|M|administrator|91344
8|36|M|administrator|05201
9|29|M|student|01002
10|53|M|lawyer|90703


#### Loading the Dataset with Pandas

Use **pandas** to load the dataset into a DataFrame for analysis. Follow these steps:  

1. Import the necessary library: `pandas`.  
2. Use `pd.read_csv()` (or an appropriate function) to read the dataset file.  
3. Ensure the dataset is loaded with the correct delimiter (e.g., `','`, `'\t'`,`'|'` , or another separator if needed).  
4. Select and display the first few rows using `.head()`.

Ensure that:  

- The `ratings` dataset is read from `"u.data"` using tab (`'\t'`) as a separator and column names (`"user_id"`, `"movie_id"`, `"rating"` and `"timestamp"`).  
- The `movies` dataset is read from `"u.item"` using `'|'` as a separator, use columns (`0`, `1`, `2`), encoding (`"latin-1"`) and name the columns (`movie_id`, `title`, and `release_date`).  
- The `users` dataset is read from `"u.user"` using `'|'` as a separator, use columns (`0`, `1`, `2`, `3`) and name the columns (`user_id`, `age`, `gender`, and `occupation`).

In [5]:
# ratings
ratings = pd.read_csv("/content/u.data", sep='\t', header=None, names=["user_id", "movie_id", "rating", "timestamp"], index_col=False)
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
# movies
labels = ["movie_id", "title", "release_date"]
movies = pd.read_csv("/content/u.item", sep='|', header=None, names=labels, index_col=False, encoding='latin-1')
movies.head()

<ipython-input-6-e06e7cc3494f>:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  movies = pd.read_csv("/content/u.item", sep='|', header=None, names=labels, index_col=False, encoding='latin-1')


,movie_id,title,release_date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995


In [7]:
# users
labels = ["user_id", "age", "gender", "occupation"]
users = pd.read_csv("/content/u.user", header=None, index_col=False, names=labels, sep='|')
users.head()

<ipython-input-7-e4aca8f13b42>:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  users = pd.read_csv("/content/u.user", header=None, index_col=False, names=labels, sep='|')


,user_id,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other


**Note:** As a **Bonus** task save the `ratings`, `movies` and `users` dataframe created into a `.csv` file format. <br>
**Hint:** Use the `to_csv()` function in pandas to save these DataFrames as CSV files.

Saving dataframes to csv

In [8]:
# ratings
ratings.to_csv("ratings.csv")

In [9]:
# movies
movies.to_csv("movies.csv")

In [10]:
# users
users.to_csv("users.csv")

**Display the first 10 rows of each file.**

In [ ]:
# ratings
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [ ]:
# movies
movies.head(10)

,movie_id,title,release_date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995
6,7,Twelve Monkeys (1995),01-Jan-1995
7,8,Babe (1995),01-Jan-1995
8,9,Dead Man Walking (1995),01-Jan-1995
9,10,Richard III (1995),22-Jan-1996


In [ ]:
# users
users.head(10)

,user_id,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other
5,6,42,M,executive
6,7,57,M,administrator
7,8,36,M,administrator
8,9,29,M,student
9,10,53,M,lawyer


### Data Cleaning and Exploration with Pandas  

After loading the dataset, it’s important to clean and explore the data to ensure consistency and accuracy. Below are key **pandas** functions for cleaning and understanding the dataset.

#### 1. Handle Missing Values  
- `df.dropna()` – Removes rows with missing values.  
- `df.fillna(value)` – Fills missing values with a specified value.  

#### 2. Remove Duplicates  
- `df.drop_duplicates()` – Drops duplicate rows from the dataset.  

#### 3. Handle Incorrect Data Types  
- `df.astype(dtype)` – Converts columns to the appropriate data type.  

#### 4. Filter Outliers (if applicable)  
- `df[df['column_name'] > threshold]` – Filters rows based on a condition.  

#### 5. Rename Columns (if needed)  
- `df.rename(columns={'old_name': 'new_name'})` – Renames columns for clarity.  

#### 6. Reset Index  
- `df.reset_index(drop=True, inplace=True)` – Resets the index after cleaning.  

### Data Exploration Functions  

To better understand the dataset, use these **pandas** functions:  

- `df.shape` – Returns the number of rows and columns in the dataset.  
- `df.nunique()` – Displays the number of unique values in each column.  
- `df['column_name'].unique()` – Returns unique values in a specific column.  

**Example Usage in Pandas:**  
```python
import pandas as pd

# Load dataset
df = pd.read_csv("your_file.csv")

# Drop missing values
df_cleaned = df.dropna()

# Remove duplicate rows
df_cleaned = df_cleaned.drop_duplicates()

# Convert 'timestamp' column to datetime format
df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'])

# Display dataset shape
print("Dataset shape:", df_cleaned.shape)

# Display number of unique values in each column
print("Unique values per column:\n", df_cleaned.nunique())

# Display unique movie IDs
print("Unique movie IDs:", df_cleaned['movie_id'].unique()[:10])  # Show first 10 unique movie IDs


**Note:** The functions mentioned above are some of the widely used **pandas** functions for data cleaning and exploration. However, it is not necessary that all of these functions will be required in the exercises below. Use them as needed based on the dataset and the specific tasks.

**Convert Timestamps into Readable dates.**

In [ ]:
# ratings
# Load dataset
ratings = pd.read_csv("ratings.csv")

# Drop missing values
ratings.dropna(inplace=True)

# Remove duplicate rows
ratings.drop_duplicates(inplace=True)
ratings.drop(columns=['Unnamed: 0'], inplace=True)

# Convert 'timestamp' column to datetime format
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'])

ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,1970-01-01 00:00:00.881250949
1,186,302,3,1970-01-01 00:00:00.891717742
2,22,377,1,1970-01-01 00:00:00.878887116
3,244,51,2,1970-01-01 00:00:00.880606923
4,166,346,1,1970-01-01 00:00:00.886397596


**Check for Missing Values**
Drop any rows with missing values

In [ ]:
# ratings
ratings = ratings.dropna()

In [ ]:
# movies
movies = movies.dropna()

In [ ]:
# users
users = users.dropna()

**Print the total number of users, movies, and ratings.**

In [ ]:
print(f"Total Users: {users.shape[0]}")
print(f"Total Movies: {movies.shape[0]}")
print(f"Total Ratings: {ratings.shape[0]}")

Total Users: 943
Total Movies: 1681
Total Ratings: 100000


## **Part 2: Collaborative Filtering-Based Recommendation**

### **Create a User-Item Matrix**

#### Instructions for Creating a User-Movie Rating Matrix

In this exercise, you will create a user-movie rating matrix using **pandas**. This matrix will represent the ratings that users have given to different movies.

1. **Dataset Overview**:  
   The dataset has already been loaded. It includes the following key columns:
   - `user_id`: The ID of the user.
   - `movie_id`: The ID of the movie.
   - `ratings`: The rating the user gave to the movie.

2. **Create the User-Movie Rating Matrix**:  
   Use the **`pivot()`** function in **pandas** to reshape the data. Your goal is to create a matrix where:
   - Each **row** represents a **user**.
   - Each **column** represents a **movie**.
   - Each **cell** contains the **rating** that the user has given to the movie.

   Specify the following parameters for the `pivot()` function:
   - **`index`**: The `user_id` column (this will define the rows).
   - **`columns`**: The `movie_id` column (this will define the columns).
   - **`values`**: The `rating` column (this will fill the matrix with ratings).

3. **Inspect the Matrix**:  
   After creating the matrix, examine the first few rows of the resulting matrix to ensure it has been constructed correctly.

4. **Handle Missing Values**:  
   It's likely that some users have not rated every movie, resulting in `NaN` values in the matrix. You will need to handle these missing values. Consider the following options:
   - **Fill with 0**: If you wish to represent missing ratings as zeros (indicating no rating).
   - **Fill with the average rating**: Alternatively, replace missing values with the average rating for each movie.

**Create the user-movie rating matrix using the `pivot()` function.**

In [ ]:
user_movie_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating')

**Display the matrix to verify the transformation.**

In [ ]:
user_movie_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **User-Based Collaborative Filtering Recommender System**

#### **Objective**
In this task, you will implement a **user-based collaborative filtering** movie recommendation system using the **Movie dataset**. The goal is to recommend movies to a user based on the preferences of similar users.

##### **Step 1: Import Required Libraries**
Before starting, ensure you have the necessary libraries installed. Use the following imports:

```python
import pandas as pd  # For handling data
import numpy as np   # For numerical computations
from sklearn.metrics.pairwise import cosine_similarity  # For computing user similarity
```

##### **Step 2: Compute User-User Similarity**
- We will use **cosine similarity** to measure how similar each pair of users is based on their movie ratings.
- Since `cosine_similarity` does not handle missing values (NaN), replace them with `0` before computation.

##### **Instructions:**
1. Fill missing values with `0` using `.fillna(0)`.
2. Compute similarity using `cosine_similarity()`.
3. Convert the result into a **Pandas DataFrame**, with users as both row and column labels.

##### **Hint:**  
You can achieve this using the following approach:

```python
user_similarity = cosine_similarity(user_movie_matrix.fillna(0))
user_sim_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)
```

##### **Step 3: Implement the Recommendation Function**
Now, implement the function `recommend_movies_for_user(user_id, num=5)` to recommend movies for a given user.

##### **Function Inputs:**
- `user_id`: The target user for whom we need recommendations.
- `num`: The number of movies to recommend (default is 5).

##### **Function Steps:**
1. Find **similar users**:
   - Retrieve the similarity scores for the given `user_id`.
   - Sort them in **descending** order (highest similarity first).
   - Exclude the user themselves.
   
2. Get the **movie ratings** from these similar users.

3. Compute the **average rating** for each movie based on these users' preferences.

4. Sort the movies in **descending order** based on the computed average ratings.

5. Retrieve the **top `num` recommended movies**.

6. Map **movie IDs** to their **titles** using the `movies` DataFrame.

7. Return the results as a **Pandas DataFrame** with rankings.

##### **Step 4: Return the Final Recommendation List**
Your function should return a **DataFrame** structured as follows:

| Ranking | Movie Name |
|---------|-----------|
| 1       | Movie A   |
| 2       | Movie B   |
| 3       | Movie C   |
| 4       | Movie D   |
| 5       | Movie E   |

##### **Hint:** Your final DataFrame should be created like this:
```python
result_df = pd.DataFrame({
    'Ranking': range(1, num+1),
    'Movie Name': movie_names     
})
result_df.set_index('Ranking', inplace=True)
```

#### **Example: User-Based Collaborative Filtering**
```python
recommend_movies_for_user(10, num = 5)
```
**Output:**
```
| Ranking | Movie Name                     |
|---------|--------------------------------|
| 1       | In the Company of Men (1997)   |
| 2       | Misérables, Les (1995)         |
| 3       | Thin Blue Line, The (1988)     |
| 4       | Braindead (1992)               |
| 5       | Boys, Les (1997)               |


## Description

➤ Fills all missing values (NaNs) in the user-movie rating matrix with 0.

➤ Calculates pairwise cosine similarity between users based on their movie ratings.

➤ Returns a square matrix where each element [i][j] indicates similarity between user i and user j.

➤ Converts the similarity matrix into a pandas DataFrame for easier readability and access.

➤ Uses the original user IDs (from user_movie_matrix) as both row and column labels.

➤ Helps in quickly inspecting how similar users are to each other.



In [ ]:
user_movie_matrix = user_movie_matrix.fillna(0)
user_similarity = cosine_similarity(user_movie_matrix)
user_sim_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)
user_movie_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


*get_sim_scores(user_sim, user_id)*

➤Retrieves similarity scores between the target user and all other users.

➤Sorts scores in descending order.

➤Removes the user’s own similarity score.

➤Returns a DataFrame with other users and their similarity scores.

-------------------------------------------------------------------------------
*get_unrated_movies(user_movie, user_id)*

➤Gets the row for the target user from the user-movie matrix.

➤Intended to return movies not rated by the user (ratings == 0).

-------------------------------------------------------------------------------


*get_user_ratings(user_movie, movie_id, user_df)*

➤For a given movie, gets ratings from similar users.

➤Merges those ratings with the similarity scores.

➤Calculates average rating (currently unweighted).

-------------------------------------------------------------------------------


*top_n_recs(user_id, num)*

➤Gets similar users using get_sim_scores.

➤Finds movies not rated by the user using get_unrated_movies.

➤Predicts ratings for each unrated movie using get_user_ratings.

➤Sorts movies by predicted rating and selects top n.

➤Merges with movies DataFrame to get titles.

➤Returns list of recommended movie titles.

-------------------------------------------------------------------------------


*print_movie_table(movies)*

➤Prints recommended movies in a simple, markdown-style table with rankings.


-------------------------------------------------------------------------------

print_movie_table(n_movies)

➤Generates top 5 recommendations for user 10 and prints them as a table.



In [ ]:
def get_sim_scores(user_sim, user_id):
  new_df = user_sim[[user_id]].sort_values(by=user_id, ascending=False)
  new_df.drop(user_id, inplace=True)
  return new_df


def get_unrated_movies(user_movie, user_id): #take look into this one
  zero_vals = [col for col in user_movie.columns if user_movie.loc[user_id, col] == 0.0]
  return zero_vals

def get_user_ratings(user_movie, movie_id, user_df):
  #for the given movie id get nonzero ratings of the users - drop any ratings that are zero
  movie_ratings = user_movie[[movie_id]]
  movie_ratings = movie_ratings[movie_ratings[movie_id] > 0.0]
  imp_movie_ratings = pd.merge(user_df, movie_ratings, on='user_id')
  avg_rating = imp_movie_ratings.iloc[:,1].sum()/movie_ratings.shape[0]
  return avg_rating

def top_n_recs(user_id, num):
  #get df containing users with sim scores >0
  user_df = get_sim_scores(user_sim_df, user_id)
  #get movies that have not been rated by user yet.
  movies_unrated = get_unrated_movies(user_movie_matrix, user_id)
  rating_preds = []
  for movie in movies_unrated:
    rating_preds.append(get_user_ratings(user_movie_matrix, movie, user_df))
  pred_df = pd.DataFrame({"movie_id": movies_unrated, "pred_rating":rating_preds})
  pred_df = pred_df.sort_values(by="pred_rating", ascending=False).iloc[:num,:] #take only the top n movies
  pred_df = pd.merge(pred_df, movies, on="movie_id", how='left')
  return list(pred_df['title'])

def print_movie_table(movies):
    headers = ['Ranking', 'Movie Name']
    rankings = list(range(1, len(movies) + 1))

    # Calculate column widths
    rank_width = max(len(str(r)) for r in rankings + [headers[0]])
    movie_width = max(len(movie) for movie in movies + [headers[1]])

    # Header and separator
    header = f"| {headers[0]:<{rank_width}} | {headers[1]:<{movie_width}} |"
    separator = f"|{'-' * (rank_width + 2)}|{'-' * (movie_width + 2)}|"

    print(header)
    print(separator)

    for rank, movie in zip(rankings, movies):
        print(f"| {rank:<{rank_width}} | {movie:<{movie_width}} |")
n_movies = top_n_recs(10, 5)
print_movie_table(n_movies)

| Ranking | Movie Name                                 |
|---------|--------------------------------------------|
| 1       | Star Kid (1997)                            |
| 2       | Marlene Dietrich: Shadow and Light (1996)  |
| 3       | Someone Else's America (1995)              |
| 4       | Prefontaine (1997)                         |
| 5       | Saint of Fort Washington, The (1993)       |


In [ ]:
n_movies = top_n_recs(34, 5)
print_movie_table(n_movies)

| Ranking | Movie Name                                        |
|---------|---------------------------------------------------|
| 1       | Entertaining Angels: The Dorothy Day Story (1996) |
| 2       | Someone Else's America (1995)                     |
| 3       | Great Day in Harlem, A (1994)                     |
| 4       | Aiqing wansui (1994)                              |
| 5       | They Made Me a Criminal (1939)                    |


## Insights:

Insights on the Code & Implementation
Similarity Retrieval (get_sim_scores):
This function retrieves and sorts the similarity scores between a target user and all other users from a precomputed similarity matrix. By dropping the target user's own score, it ensures recommendations aren't biased by self-comparison.

Unrated Movie Identification (get_unrated_movies):
The function scans the user-movie matrix to find movies that the target user has not rated (i.e., those with a rating of 0.0). This is essential to avoid recommending already watched movies.

Rating Estimation (get_user_ratings):
For each unrated movie, this function gathers non-zero ratings from similar users and merges them with their similarity scores. Then, it calculates a simple average rating to estimate how much the target user might like the movie. While functional, this could be improved by using a weighted average based on similarity scores for more accurate predictions.

Top-N Recommendations (top_n_recs):
This function brings everything together: it filters out unrated movies, predicts scores for each based on similar users, and returns the top n movie titles by highest predicted rating. It's a clean and understandable approach but computationally intensive for large datasets due to looping over each unrated movie.

### **Item-Based Collaborative Filtering Recommender System**

#### **Objective**
In this task, you will implement an **item-based collaborative filtering** recommendation system using the **Movie dataset**. The goal is to recommend movies similar to a given movie based on user rating patterns.

#### **Step 1: Import Required Libraries**
Although we have done this part already in the previous task but just to emphasize the importance reiterrating this part.

Before starting, ensure you have the necessary libraries installed. Use the following imports:

```python
import pandas as pd  # For handling data
import numpy as np   # For numerical computations
from sklearn.metrics.pairwise import cosine_similarity  # For computing item similarity
```

#### **Step 2: Compute Item-Item Similarity**
- We will use **cosine similarity** to measure how similar each pair of movies is based on their user ratings.
- Since `cosine_similarity` does not handle missing values (NaN), replace them with `0` before computation.
- Unlike user-based filtering, we need to **transpose** (`.T`) the `user_movie_matrix` because we want similarity between movies (columns) instead of users (rows).

##### **Instructions:**
1. Transpose the user-movie matrix using `.T` to make movies the rows.
2. Fill missing values with `0` using `.fillna(0)`.
3. Compute similarity using `cosine_similarity()`.
4. Convert the result into a **Pandas DataFrame**, with movies as both row and column labels.

##### **Hint:**  
You can achieve this using the following approach:

```python
item_similarity = cosine_similarity(user_movie_matrix.T.fillna(0))
item_sim_df = pd.DataFrame(item_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)
```

#### **Step 3: Implement the Recommendation Function**
Now, implement the function `recommend_movies(movie_name, num=5)` to recommend movies similar to a given movie.

##### **Function Inputs:**
- `movie_name`: The target movie for which we need recommendations.
- `num`: The number of similar movies to recommend (default is 5).

##### **Function Steps:**
1. Find the **movie_id** corresponding to the given `movie_name` in the `movies` DataFrame.
2. If the movie is not found, return an appropriate message.
3. Extract the **similarity scores** for this movie from `item_sim_df`.
4. Sort the movies in **descending order** based on similarity (excluding the movie itself).
5. Retrieve the **top `num` similar movies**.
6. Map **movie IDs** to their **titles** using the `movies` DataFrame.
7. Return the results as a **Pandas DataFrame** with rankings.

#### **Step 4: Return the Final Recommendation List**
Your function should return a **DataFrame** structured as follows:

| Ranking | Movie Name |
|---------|-----------|
| 1       | Movie A   |
| 2       | Movie B   |
| 3       | Movie C   |
| 4       | Movie D   |
| 5       | Movie E   |

##### **Hint:** Your final DataFrame should be created like this:
```python
result_df = pd.DataFrame({
    'ranking': range(1, num+1),
    'movie_name': movie_names
})
result_df.set_index('ranking', inplace=True)
```

#### **Example: Item-Based Collaborative Filtering**
```python
recommend_movies("Jurassic Park (1993)", num=5)
```
**Output:**
```
| Ranking | Movie Name                               |
|---------|------------------------------------------|
| 1       | Top Gun (1986)                           |
| 2       | Empire Strikes Back, The (1980)          |
| 3       | Raiders of the Lost Ark (1981)           |
| 4       | Indiana Jones and the Last Crusade (1989)|
| 5       | Speed (1994)                             |


*cosine_similarity(user_movie_matrix.T.fillna(0))*

➤ Transposes the user-movie matrix so that movies become rows.

➤ Fills missing values with 0 (assumes unrated = 0).

➤ Computes cosine similarity between movies based on user ratings.

------------------------------------------------------------------------------

*item_sim_df = pd.DataFrame(...)*

➤ Converts the similarity matrix into a pandas DataFrame.

➤ Uses movie IDs as both row and column labels for easier lookup.

------------------------------------------------------------------------------

*get_n_movies(movie_name, num)*

➤ Searches for the movie_id corresponding to the given movie_name.

➤ If not found, returns "Movie not found".

➤ Retrieves the similarity scores between the given movie and all others.

➤ Sorts similar movies in descending order (most similar at the top).

➤ Drops the movie itself from the list (since it's 100% similar to itself).

➤ Merges with movies DataFrame to get movie titles.

➤ Returns a list of the top num similar movie titles.


------------------------------------------------------------------------------
*movie_list = get_n_movies("Jurassic Park (1993)", 5)*

*print_movie_table(movie_list)*

➤ Gets the top 5 most similar movies to Jurassic Park (1993).

➤ Prints them in a clean, ranked markdown-style table.

In [ ]:
# Code the function here
item_similarity = cosine_similarity(user_movie_matrix.T.fillna(0))
item_sim_df = pd.DataFrame(item_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)

def get_n_movies(movie_name, num):
  # movies
  try:
    movie_id = movies[movies['title']== movie_name].iloc[0,0]
  except IndexError:
    return 'Movie not found'
  movie_df = item_sim_df[[movie_id]].sort_values(by=movie_id, ascending=False).iloc[:num+1,:]
  movie_df.drop(movie_id, inplace=True)
  movie_df = pd.merge(movie_df, movies, on="movie_id", how='left')
  return list(movie_df['title'])

movie_list = get_n_movies("Speed (1994)", 5)
print_movie_table(movie_list)

| Ranking | Movie Name                        |
|---------|-----------------------------------|
| 1       | Jurassic Park (1993)              |
| 2       | True Lies (1994)                  |
| 3       | Top Gun (1986)                    |
| 4       | Fugitive, The (1993)              |
| 5       | Terminator 2: Judgment Day (1991) |


## Insights

Movie Lookup (get_n_movies):
This function takes a movie title and retrieves its corresponding movie_id from the movies DataFrame. It uses a try-except block to handle cases where the movie isn't found, returning a helpful error message if needed.

Similarity Retrieval:
Once the movie_id is found, the function fetches its similarity scores from the item_sim_df, a precomputed item-item cosine similarity matrix. It sorts these scores in descending order to prioritize the most similar movies.

Filtering & Cleaning:
The top num + 1 most similar movies are selected initially to account for the movie itself (which will always have a similarity score of 1). Then, the movie itself is dropped from the list to avoid self-recommendation.

Mapping & Return:
The remaining movie_ids are merged back with the movies DataFrame to fetch their corresponding titles. The function returns a clean list of movie titles as recommendations.

## **Part 3: Graph-Based Recommender (Pixie-Inspired Algorithm)**

### **Adjacency List**

#### **Objective**
In this task, you will preprocess the Movie dataset and construct a **graph representation** where:
- **Users** are connected to the movies they have rated.
- **Movies** are connected to users who have rated them.
  
This graph structure will help in exploring **user-movie relationships** for recommendations.

#### **Step 1: Merge Ratings with Movie Titles**
Since we have **movie IDs** in the ratings dataset but need human-readable movie titles, we will:
1. Merge the `ratings` DataFrame with the `movies` DataFrame using the `'movie_id'` column.
2. This allows each rating to be associated with a **movie title**.

#### **Hint:**
Use the following Pandas operation to merge:
```python
ratings = ratings.merge(movies, on='movie_id')
```


#### **Step 2: Aggregate Ratings**
Since multiple users may rate the same movie multiple times, we:
1. Group the dataset by `['user_id', 'movie_id', 'title']`.
2. Compute the **mean rating** for each movie by each user.
3. Reset the index to ensure we maintain a clean DataFrame structure.

#### **Hint:**  
Use `groupby()` and `mean()` as follows:
```python
ratings = ratings.groupby(['user_id', 'movie_id', 'title'])['rating'].mean().reset_index()
```

#### **Step 3: Normalize Ratings**
Since different users have different rating biases, we normalize ratings by:
1. **Computing each user's mean rating**.
2. **Subtracting the mean rating** from each individual rating.

#### **Instructions:**
- Use `groupby('user_id')` to group ratings by users.
- Apply `transform(lambda x: x - x.mean())` to adjust ratings.

#### **Hint:**  
Normalize ratings using:
```python
ratings['rating'] = ratings.groupby('user_id')['rating'].transform(lambda x: x - x.mean())
```
This ensures each user’s ratings are centered around zero, making similarity calculations fairer.

#### **Step 4: Construct the Graph Representation**
We represent the user-movie interactions as an **undirected graph** using an **adjacency list**:
- Each **user** is a node connected to movies they rated.
- Each **movie** is a node connected to users who rated it.

#### **Graph Construction Steps:**
1. Initialize an empty dictionary `graph = {}`.
2. Iterate through the **ratings dataset**.
3. For each `user_id` and `movie_id` pair:
   - Add the movie to the user’s set of connections.
   - Add the user to the movie’s set of connections.

#### **Hint:**  
The following code builds the graph:

```python
graph = {}
for _, row in ratings.iterrows():
    user, movie = row['user_id'], row['movie_id']
    if user not in graph:
        graph[user] = set()
    if movie not in graph:
        graph[movie] = set()
    graph[user].add(movie)
    graph[movie].add(user)
```

This results in a **bipartite graph**, where:
- **Users** are connected to multiple movies.
- **Movies** are connected to multiple users.

#### **Step 5: Understanding the Graph**
- **Nodes** in the graph represent **users and movies**.
- **Edges** exist between a user and a movie **if the user has rated the movie**.
- This structure allows us to find **users with similar movie tastes** and **movies frequently watched together**.

#### **Exploring the Graph**
- **Find a user’s rated movies:**  
  ```python
  user_id = 1
  print(graph[user_id])  # Movies rated by user 1
  ```

- **Find users who rated a movie:**  
  ```python
  movie_id = 50
  print(graph[movie_id])  # Users who rated movie 50
  ```

In [ ]:
# Code the function here
ratings = ratings.merge(movies, on='movie_id')
ratings = ratings.groupby(['user_id', 'movie_id', 'title'])['rating'].mean().reset_index()
ratings['rating'] = ratings.groupby('user_id')['rating'].transform(lambda x: x - x.mean())

In [ ]:
#need separate graphs to keep track of movies and users connections since there is no way to distinguish between them
graph = {}
user_graph = {}
movie_graph = {}
for _, row in ratings.iterrows():
    user, movie = row['user_id'], row['movie_id']
    if user not in graph:
        graph[user] = set()
    if user not in user_graph:
        user_graph[user] = set()
    if movie not in graph:
        graph[movie] = set()
    if movie not in movie_graph:
        movie_graph[movie] = set()
    graph[user].add(movie)
    graph[movie].add(user)
    user_graph[user].add(movie)
    movie_graph[movie].add(user)


In [ ]:
movie_id = 50
print(movie_graph[movie_id])  # Users who rated movie 50

user_id = 1
print(graph[user_id])  # Movies rated by user 1

{1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 18, 20, 21, 22, 23, 25, 26, 27, 28, 30, 32, 37, 41, 42, 43, 44, 45, 46, 48, 49, 51, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 77, 79, 80, 82, 83, 85, 87, 89, 91, 92, 94, 95, 96, 97, 99, 101, 102, 103, 104, 108, 109, 113, 115, 116, 117, 119, 120, 121, 123, 124, 125, 127, 128, 130, 132, 137, 141, 144, 145, 148, 150, 151, 153, 154, 157, 158, 160, 161, 162, 169, 174, 175, 176, 177, 178, 182, 183, 184, 185, 188, 189, 192, 194, 197, 198, 200, 201, 203, 209, 210, 213, 214, 215, 216, 217, 221, 222, 227, 230, 231, 232, 233, 234, 235, 236, 239, 244, 245, 246, 247, 248, 249, 250, 251, 253, 254, 256, 257, 262, 263, 265, 267, 268, 269, 270, 271, 272, 274, 275, 276, 277, 279, 280, 283, 286, 287, 288, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 301, 303, 305, 307, 308, 310, 311, 312, 313, 316, 318, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 332, 334, 336, 337, 339, 340, 343, 344, 345, 346, 347, 350, 352, 354,

### **Implement Weighted Random Walks**

#### **Random Walk-Based Movie Recommendation System (Weighted Pixie)**

#### **Objective**
In this task, you will implement a **random-walk-based recommendation algorithm** using the **Weighted Pixie** method. This technique uses a **user-movie bipartite graph** to recommend movies by simulating a random walk from a given user or movie.

#### **Step 1: Import Required Libraries**
Make sure you have the necessary libraries:

```python
import random  # For random walks
import pandas as pd  # For handling data
```

#### **Step 2: Implement the Random Walk Algorithm**
Your task is to **simulate a random walk** from a given starting point in the **bipartite user-movie graph**.

##### **Hints for Implementation**
- Start from **either a user or a movie**.
- At each step, **randomly move** to a connected node.
- Keep track of **how many times each movie is visited**.
- After completing the walk, **rank movies by visit count**.

#### **Step 3: Implement User-Based Recommendation**
**Hints:**
- Check if the `user_id` exists in the `graph`.
- Start a loop that runs for `walk_length` steps.
- Randomly pick a **connected node** (user or movie).
- Track how many times each **movie** is visited.
- Sort movies by visit frequency and return the **top N**.

#### **Step 4: Implement Movie-Based Recommendation**
**Hints:**
- Find the `movie_id` corresponding to the given `movie_name`.
- Ensure the movie exists in the `graph`.
- Start a random walk from that movie.
- Follow the same **tracking and ranking** process as the user-based version.

**Note:**  
**Your task:** Implement a function `weighted_pixie_recommend(user_id, walk_length=15, num=5)` or `weighted_pixie_recommend(movie_name, walk_length=15, num=5)`.  
**Implement either Step 3 or Step 4.**

In [ ]:
import random  # For random walks
import pandas as pd  # For handling data

#### **Step 5: Running Your Recommendation System**
Once your function is implemented, test it by calling:

##### **Example: User-Based Recommendation**
```python
weighted_pixie_recommend(1, walk_length=15, num=5)
```
| Ranking | Movie Name                     |
|---------|--------------------------------|
| 1       | My Own Private Idaho (1991)   |
| 2       | Aladdin (1992)                |
| 3       | 12 Angry Men (1957)           |
| 4       | Happy Gilmore (1996)          |
| 5       | Copycat (1995)                |


##### **Example: Movie-Based Recommendation**
```python
weighted_pixie_recommend("Jurassic Park (1993)", walk_length=10, num=5)
```
| Ranking | Movie Name                           |
|---------|-------------------------------------|
| 1       | Rear Window (1954)                 |
| 2       | Great Dictator, The (1940)         |
| 3       | Field of Dreams (1989)             |
| 4       | Casablanca (1942)                  |
| 5       | Nightmare Before Christmas, The (1993) |


#### **Step 6: Understanding the Results**
Your function should return a **DataFrame** structured as follows:

| Ranking | Movie Name |
|---------|-----------|
| 1       | Movie A   |
| 2       | Movie B   |
| 3       | Movie C   |
| 4       | Movie D   |
| 5       | Movie E   |

Each movie is ranked based on **how frequently it was visited** during the walk.

#### **Experiment with Different Parameters**
- Try different **`walk_length`** values and observe how it changes recommendations.
- Adjust the number of recommended movies (`num`).

In [ ]:
import random  # For random walks
import pandas as pd  # For handling data

In [ ]:
def weighted_pixie_recommend(user_id, walk_length=15, num=5):
  is_user = 1
  if user_id in graph:
    # Start a loop that runs for walk_length steps.
    visited = {i: 0 for i in movie_graph.keys()}  #setting default frequency for all movies to 0
    for step in range(walk_length):
      options = list(graph[user_id])
      #select one of the paths to follow
      nextstep = random.choice(options)
      if is_user == -1 :
        visited[nextstep] += 1 #only keep track of frequency of movies and not of users
      is_user = is_user * -1
      user_id = nextstep
    sorted_items = sorted(visited.items(), key=lambda item: item[1], reverse=True)
    sorted_keys = [item[0] for item in sorted_items]
    movie_names = []
    for movie_id in sorted_keys[:num]:
      movie_names.append(movies[movies['movie_id'] == movie_id].iloc[0,1])
    movie_df = pd.DataFrame({
      'Ranking': range(1, num+1),
      'Movie Name': movie_names
    })
    movie_df.set_index('Ranking', inplace=True)
    return movie_df
  else:
    return 'User not found'

weighted_pixie_recommend(4)

,Movie Name
Ranking,
1,"Crossing Guard, The (1995)"
2,Bonnie and Clyde (1967)
3,"Fan, The (1996)"
4,"Juror, The (1996)"
5,Tomorrow Never Dies (1997)


In [ ]:
weighted_pixie_recommend(20)

,Movie Name
Ranking,
1,Indiana Jones and the Last Crusade (1989)
2,Star Trek: First Contact (1996)
3,L.A. Confidential (1997)
4,Jackie Brown (1997)
5,Batman (1989)


## Insights

Graph Traversal Logic:
The algorithm uses a bipartite graph (users and movies as nodes, connected by ratings) to simulate a random walk. Starting from a user, it alternates steps between user and movie nodes, mimicking how a user might explore the network of content via shared preferences.

Random Walk Steps:
The walk performs a fixed number of steps (walk_length), randomly hopping from user to movie and back. Every time a movie node is visited, a counter in the visited dictionary is incremented. This builds a frequency map of how often each movie was encountered during the simulation.

Recommendation Ranking:
After the walk, the movies are ranked by how frequently they were visited. The idea is that frequently visited movies are more connected to the target user's interests within the graph, making them more relevant.

Personalization:
The method captures indirect relationships between users and movies, which traditional collaborative filtering might miss—especially useful when direct user similarities are sparse or unavailable.

Effect of walk_length on Recommendations

*Short Walks (walk_length ~ 5–10):*

Tend to stay close to the user’s immediate neighborhood in the graph.

Recommendations are more similar to movies the user already rated (less diversity).

Fast to compute but may lack discovery of new or less obvious suggestions.

*Moderate Walks (walk_length ~ 15–25):*

Balance between relevance and exploration.

Captures both direct and slightly indirect interests.

Tends to provide better contextual recommendations and aligns well with systems like Pinterest’s Pixie algorithm.

*Long Walks (walk_length ~ 30 or more):*

Explore deeper into the graph.

Can surface diverse or unexpected recommendations, increasing serendipity.

However, may reduce precision by drifting away from core user interests.

Also more computationally expensive and potentially noisy.

Pro Tip: You can dynamically adjust walk length based on user engagement — e.g., longer walks for users with fewer interactions to promote discovery, shorter for those with strong preference patterns.

General Observations and Strengths

No Need for Explicit Similarity Metrics:
Unlike collaborative filtering, this model doesn’t require precomputed similarity matrices—it learns structure on the fly via the graph.

Adaptable to Real-Time Systems:
With optimizations, this walk-based method can be highly scalable and responsive, making it suitable for real-time applications like personalized feeds and content suggestions.

Captures Indirect Signals:
Through repeated hops, it uncovers indirect but meaningful relationships (e.g., users who didn’t rate the same movie but liked movies liked by similar users).

Limitations and Considerations

Randomness:
Recommendations can vary between runs due to the random nature of path selection—this can be good for novelty, but bad for consistency unless controlled (e.g., by seeding or averaging multiple walks).

Cold Start:
New users or movies with no connections still face the cold-start issue unless bootstrap techniques or hybrid models are added.

Unweighted Walks:
All edges are currently treated equally. Introducing weighted edges based on rating strength or recency could improve relevance by guiding the walk more meaningfully.



---

## **Submission Requirements:**

To successfully complete this assignment, ensure that you submit the following:


### **1. Jupyter Notebook Submission**
- Submit a **fully completed Jupyter Notebook** that includes:
  - **All implemented recommendation functions** (user-based, item-based, and random walk-based recommendations).
  - **Code explanations** in markdown cells to describe each step.
  - **Results and insights** from running your recommendation models.


### **2. Explanation of Pixie-Inspired Algorithms (3-5 Paragraphs)**
- Write a **detailed explanation** of **Pixie-inspired random walk algorithms** used for recommendations.
- Your explanation should cover:
  - What **Pixie-inspired recommendation systems** are.
  - How **random walks** help in identifying relevant recommendations.
  - Any real-world applications of such algorithms in industry.


### **3. Report for the Submitted Notebook**
Your report should be structured as follows:

#### **Title: Movie Recommendation System Report**

#### **1. Introduction**
- Briefly introduce **movie recommendation systems** and why they are important.
- Explain the **different approaches used** (user-based, item-based, random-walk).

#### **2. Dataset Description**
- Describe the **MovieLens 100K dataset**:
  - Number of users, movies, and ratings.
  - What features were used.
  - Any preprocessing performed.

#### **3. Methodology**
- Explain the three recommendation techniques implemented:
  - **User-based collaborative filtering** (how user similarity was calculated).
  - **Item-based collaborative filtering** (how item similarity was determined).
  - **Random-walk-based Pixie algorithm** (why graph-based approaches are effective).
  
#### **4. Implementation Details**
- Discuss the steps taken to build the functions.
- Describe how the **adjacency list graph** was created.
- Explain how **random walks** were performed and how visited movies were ranked.

#### **5. Results and Evaluation**
- Present **example outputs** from each recommendation approach.
- Compare the different methods in terms of accuracy and usefulness.
- Discuss any **limitations** in the implementation.

#### **6. Conclusion**
- Summarize the key takeaways from the project.
- Discuss potential improvements (e.g., **hybrid models, additional features**).
- Suggest real-world applications of the methods used.

### **Submission Instructions**

- Submit `.zip` file consisting of Jupyter Notebook and all the datafiles (provided) and the ones saved [i.e. `users.csv`, `movies.csv` and `ratings.csv`]. Also, include the Report and Pixie Algorithm explanation document.
- [`Bonus 10 Points`] **Upload your Jupyter Notebook, Explanation Document, and Report** to your GitHub repository.
- Ensure the repository is public and contains:
  - `users.csv`, `movies.csv` and `ratings.csv` [These are the Dataframes which were created in part 1. Save and export them as a `.csv` file]
  - `Movie_Recommendation.ipynb`
  - `Pixie_Algorithm_Explanation.pdf` or `.md`
  - `Recommendation_Report.pdf` or `.md`
- **Submit the GitHub repository link in the cell below.**


#### **Example Submission Format**
```text
GitHub Repository: https://github.com/username/Movie-Recommendation
```

In [ ]:
# Submit the Github Link here:


### **Grading Rubric: ITCS 6162 - Data Mining Assignment**


| **Category**                              | **Criteria**                                                     | **Points** |
|-------------------------------------------|----------------------------------------------------------------|------------|
| **Part 1: Exploring and Cleaning Data (15 pts)**  | Properly loads `u.user`, `u.movies`, and `u.item` datasets into DataFrames | 5 |
|                                           | Handles missing values, duplicates, and inconsistencies appropriately | 5 |
|                                           | Saves the cleaned datasets into CSV files: `users.csv`, `movies.csv`, `ratings.csv` | 5 |
| **Part 2: Collaborative Filtering-Based Recommendation (30 pts)** | Implements user-based collaborative filtering correctly | 10 |
|                                           | Implements item-based collaborative filtering correctly | 10 |
|                                           | Computes similarity measures accurately and provides valid recommendations | 10 |
| **Part 3: Graph-Based Recommender (Pixie-Inspired Algorithm) (35 pts)** | Constructs adjacency lists properly from user-movie interactions | 10 |
|                                           | Implements weighted random walk-based recommendation correctly | 15 |
|                                           | Explains and justifies the algorithm design choices (Pixie-inspired) | 10 |
| **Code Quality & Documentation (10 pts)** | Code is well-structured, efficient, and follows best practices | 5 |
|                                           | Markdown explanations and comments are clear and enhance understanding | 5 |
| **Results & Interpretation (5 pts)**      | Provides meaningful insights from the recommendation system's output | 5 |
| **Submission & Report (5 pts)**          | Submits all required files in the correct format (ZIP file with Jupyter notebook, processed CSV files, and project report) | 5 |
| **Total**                                 |                              | 100 |

#### **Bonus (10 pts)**
| **Category**                              | **Criteria**                                                     | **Points** |
|-------------------------------------------|----------------------------------------------------------------|------------|
| **GitHub Submission**                     | Provides a well-documented GitHub repository with CSV files, a structured README, and a properly formatted Jupyter Notebook | 10 |